In [1]:
import tensorflow as tf
import pandas as pd

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names, na_values='?', comment='\t',sep=' ', skipinitialspace=True)

dataset = raw_dataset.copy()
dataset = dataset.dropna()
dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [2]:
for col in dataset.columns:
    dataset[col] = dataset[col].astype(dtype='float32')

dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MPG           392 non-null    float32
 1   Cylinders     392 non-null    float32
 2   Displacement  392 non-null    float32
 3   Horsepower    392 non-null    float32
 4   Weight        392 non-null    float32
 5   Acceleration  392 non-null    float32
 6   Model Year    392 non-null    float32
 7   Origin        392 non-null    float32
dtypes: float32(8)
memory usage: 15.3 KB


In [3]:
from sklearn.model_selection import train_test_split

X = dataset.drop(columns=['Origin'])
y = dataset['Origin']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(352, 7)
(40, 7)
(352,)
(40,)


In [4]:
tf.random.set_seed(42)

norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])

model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1)
])

### Custom Loss Functions

In [60]:
# assume keras does not suppport hubar_loss function

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error)<1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [6]:
model.compile(
    loss=huber_fn,
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[huber_fn]
)

norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=3, validation_split=0.1)

Epoch 1/3
10/10 [==============================] - 2s 31ms/step - loss: 0.9251 - huber_fn: 0.9251 - val_loss: 0.6404 - val_huber_fn: 0.6404
Epoch 2/3
10/10 [==============================] - 0s 5ms/step - loss: 0.5403 - huber_fn: 0.5403 - val_loss: 0.3204 - val_huber_fn: 0.3204
Epoch 3/3
10/10 [==============================] - 0s 5ms/step - loss: 0.2808 - huber_fn: 0.2808 - val_loss: 0.2748 - val_huber_fn: 0.2748


### Saving and Loading Models that Contain Custom Components

In [7]:
model.save("my_custom_model_with_custom_loss_fn")

INFO:tensorflow:Assets written to: my_custom_model_with_custom_loss_fn\assets


INFO:tensorflow:Assets written to: my_custom_model_with_custom_loss_fn\assets


In [8]:
model = tf.keras.models.load_model("my_custom_model_with_custom_loss_fn", custom_objects={"huber_fn":huber_fn})

In [9]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.2922 - huber_fn: 0.2922


[0.2922378182411194, 0.2922378182411194]

With the current implementation, any error between -1 and 1 is considered "small". But what if you want a different threshold?

In [10]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=0.1, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2/2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [11]:
model.compile(
    loss=HuberLoss(0.2),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3)
)

norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=3, validation_split=0.1)

Epoch 1/3
10/10 [==============================] - 1s 22ms/step - loss: 0.0820 - val_loss: 0.0956
Epoch 2/3
10/10 [==============================] - 0s 5ms/step - loss: 0.0765 - val_loss: 0.0939
Epoch 3/3
10/10 [==============================] - 0s 5ms/step - loss: 0.0737 - val_loss: 0.0919


In [12]:
model.save("my_custom_model_with_custom_loss_class")

INFO:tensorflow:Assets written to: my_custom_model_with_custom_loss_class\assets


INFO:tensorflow:Assets written to: my_custom_model_with_custom_loss_class\assets


In [13]:
model = tf.keras.models.load_model("my_custom_model_with_custom_loss_class", custom_objects={"HuberLoss":HuberLoss})

In [14]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.0897


0.08967692404985428

### Custom Activation Functions, Initializers, Regulaizers, and Constraints

In [15]:
# activation function

def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

In [16]:
# initializer

def my_golorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / shape[0] + shape[1])

In [17]:
# l1 regulizer

def my_l1_regulizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [18]:
# constraint

def my_positive_weights(weights):
    return tf.where(weights<0. , tf.zeros_like(weights), weights)

In [19]:
# using custom components

layer = tf.keras.layers.Dense(
    1,
    activation=my_softplus,
    kernel_initializer=my_golorot_initializer,
    kernel_regularizer=my_l1_regulizer,
    kernel_constraint=my_positive_weights
)

To save the hyperparameter, you'll need to create subclass. No need to call parent in get_config function as they're not defined in parent class.

In [20]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor*weights))
    
    def get_config(self):
        return {"factor": self.factor}

### Custom Metrics
Losses and metrics are not the same thing conceptually. But in most cases we can use loss class/function as our custom metric.

In [59]:
def create_huber(threshold=0.1):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error)<1
        squared_loss = tf.square(error)/2
        linear_loss = tf.abs(error) - 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [47]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=3, validation_split=0.1)

Epoch 1/3
10/10 [==============================] - 2s 24ms/step - loss: 0.3488 - huber_fn: 0.1682 - val_loss: 0.4513 - val_huber_fn: 0.2171
Epoch 2/3
10/10 [==============================] - 0s 6ms/step - loss: 0.3356 - huber_fn: 0.1627 - val_loss: 0.4477 - val_huber_fn: 0.2151
Epoch 3/3
10/10 [==============================] - 0s 7ms/step - loss: 0.3245 - huber_fn: 0.1577 - val_loss: 0.4511 - val_huber_fn: 0.2158


In [27]:
precision = tf.keras.metrics.Precision()
precision([0,1,1,1,0,1,0,1] , [1,1,0,1,0,1,0,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [28]:
precision([0,1,0,0,1,0,1,1] , [1,0,1,1,0,0,0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [30]:
# overall precision, not batch precision
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [31]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [34]:
precision.reset_states()
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [61]:
# custom metric

class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
        
    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold" : self.threshold}

In [62]:
model.compile(loss="mse", optimizer="nadam", metrics=[HuberMetric(2.0)])

norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=3, validation_split=0.1)

Epoch 1/3
10/10 [==============================] - 2s 24ms/step - loss: 0.3197 - huber_metric_9: 0.1554 - val_loss: 0.4250 - val_huber_metric_9: 0.2050
Epoch 2/3
10/10 [==============================] - 0s 6ms/step - loss: 0.3080 - huber_metric_9: 0.1503 - val_loss: 0.4208 - val_huber_metric_9: 0.2028
Epoch 3/3
10/10 [==============================] - 0s 5ms/step - loss: 0.2994 - huber_metric_9: 0.1462 - val_loss: 0.4259 - val_huber_metric_9: 0.2042
